# Discovery

Invoke the `stac-client search` tool as a Common Workflow Language CommandLineTool and execute it with a CWL runner.

This notebook is linked to: https://eoap.github.io/zarr-cloud-native-format/cwl-cli/discovery/



## Setup

In [ ]:
export WORKSPACE=/workspace/zarr-cloud-native-format
export RUNTIME=${WORKSPACE}/runs
mkdir -p ${RUNTIME}
cd ${RUNTIME}

curl -L https://github.com/eoap/zarr-cloud-native-format/releases/download/0.5.0/app-water-bodies.0.5.0.cwl > ${WORKSPACE}/cwl-workflow/app-water-bodies.cwl 2> /dev/null

## Run the discovery step

Inspect and use `cwltool` to run the CommandLineTool definition:

The CWL document below shows the `stac-client search` step named `discovey`:

In [ ]:
cat ${WORKSPACE}/cwl-workflow/app-water-bodies.cwl | yq e '.["$graph"][0].steps.discovery' -

We can extract inspect the CWL with:

In [ ]:

cwl_url=$(cat ${WORKSPACE}/cwl-workflow/app-water-bodies.cwl | yq e '.["$graph"][0].steps.discovery.run' -)
curl -L ${cwl_url} | yq -

Run the CWL description for the `stac-client search` command line tool wrapped in a CWL CommandLineTool description:

In [ ]:
cat <<'EOF' > discovery-params.yaml
bands:
- green
- nir
search_request:
  bbox:
  - -121.399
  - 39.834
  - -120.74
  - 40.472
  collections:
  - sentinel-2-l2a
  datetime_interval:
    end:
      value: '2021-08-01T23:59:59'
    start:
      value: '2021-06-01T00:00:00'
  limit: 20
  max-items: 10
api_endpoint:
  headers: []
  url:
    value: https://earth-search.aws.element84.com/v1/
EOF

cat discovery-params.yaml | yq .

In [ ]:


cwltool \
    --podman \
    --outdir ${WORKSPACE}/runs \
    ${cwl_url} \
    discovery-params.yaml > discovery-results.json 2> discovery.log

Let's look at the content of the stderr:

In [ ]:
cat discovery.log | egrep -v "WARNING|JSHINT"

Let's inspect the stdout produced. The `output` block with the id `search_output` of the CWL description is a file:

In [ ]:
cat discovery-results.json | jq . -

List the discovered STAC Items `id`s:

In [ ]:
cat $( cat discovery-results.json | jq -r .search_output.path - ) | jq -r .features[].id -